<a href="https://colab.research.google.com/github/SURESHBEEKHANI/Autoencoders/blob/main/Vanilla%20Autoencoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Autoencoders

**Autoencoders** are a particular type of neural network, just like classifiers. Autoencoders are similar to classifiers in the sense that they compress data. However, where classifiers condense all the data of an image into a single label, autoencoders compress the data into a **latent vector**, often denoted $z$ in literature, with the goal of preserving the opportunity to recreate the exact same image in the future. Because autoencoders learn representations instead of labels, autoencoders belong to representation learning, a subfield of machine learning, but not necessarily deep learning.

While recreating the same data from a compressed version might seem like an impossible task. However, _you_ can actually do the same. You probably have no difficulty memorizing the following sequence:

$$1, 3, 5, 7, 9, 11, 13, 15, 17, 19, 21, 23, 25, 27...$$

I bet you haven't looked at every item, but you can still write down the sequence perfectly because you recognized a pattern: all uneven numbers, starting from 1.

This is what autoencoders do: they find patterns in data.

## Architecture
Autoencoders consist of two networks:

* Encoder
* Decoder

The goal of the **encoder** is to compress an image, video, or any piece of data that can be represented as a tensor, into a _latent vector_. The **decoder** does, as you might have guessed, the exact opposite.

To maximize performance, minimize the loss that is, encoders and decoders are typically symmetrical together. Naturally, the input size is equal to the output size of an autoencoder.

Autoencoders always have less input neurons in the middle layer than in the input and output layer. This is called the **bottleneck**. If it weren't for this bottleneck, the autoencoders could just copy this data over from the input to the output layer without compressing it.

![](https://upload.wikimedia.org/wikipedia/commons/2/28/Autoencoder_structure.png) [source](https://en.wikipedia.org/wiki/File:Autoencoder_structure.png)

## Training

Encoders and decoders _can_ be trained separately, but usually they are trained in one go. In order to do so, one stacks the coders together in one **stacked autoencoder**.

If one desires to train autoencoders separately, one starts by using the first hidden layer, discaring every other layer, except for the input and output layers of course. He uses the original training data at this point. Next, he uses the latent vector $z$ learnt by this mini-autoencoder and trains another autoencoder in the same way, treating the latent vectors as original data. Once the desired depth is reached, one can stack all output layers, which provided the latent vectors, together in a sinle encoder. This approach is not used in practise a lot, but literature might refer to it as greedy layerwise training so it's good to know what it means.

## Appliciations

While the phase "finding patterns" might not seem very interesting, there are a lot of exciting applications of autoencoders. We will look at three of those today:

1. Dense autoencoder: compressing data.
2. Convolutional autoencoder: a building block of DCGANs, self-supervised learning.
3. Denoising autoencoder: removing noise from poor training data.

While all of these applications use pattern finding, they have different use cases making autoencoders one of the most exciting topics of machine learning.

In [ ]:
from tensorflow import keras
from tensorflow.keras.optimizers import Adam

## Loading the data

We will load MNIST, but without labels because representation learning is **unsupervised**, or **self-supervised** which is the prefered

In [ ]:
#load the dataset not use label in  we prisuction data we Generation
(x_train, _),(x_test, _)=keras.datasets.mnist.load_data()

In [ ]:
#normilize the data range into [0,1]
x_train=x_train/255.0
x_test=x_test/255.0

## A simple autoencoder

Let's start by looking at the simplest possible autoencoder.

The `encoder` is a sequential neural network with $28 \times 28$ input neurons, $100$ neurons in the second layer and $30$ in the third. The third layer is called the "bottleneck". Feel free to play around with this variable to see how it affects results.

In [ ]:
# Define the simple autoencoder

# Create the encoder part of the autoencoder
encoder = keras.Sequential([
    # Flatten the input image of shape 28x28 into a 1D array of 784 elements
    keras.layers.Flatten(input_shape=[28, 28]),

    # First dense layer with 100 neurons and ReLU activation function
    keras.layers.Dense(100, activation='relu'),

    # Second dense layer with 30 neurons and ReLU activation function
    keras.layers.Dense(30, activation='relu')
])


In [ ]:
#create the decoder part of autoencoder
decoder = keras.Sequential([

    # Second dense layer with 100 neurons and ReLU activation function
    keras.layers.Dense(100, activation='relu', input_shape=[30]),
   # Flatten the output of the decoder into a 28x28 image
   # First dense layer with 784 neurons and sigmoid activation function
    keras.layers.Dense(784, activation='sigmoid'),
    #Reshape the image
    keras.layers.Reshape([28, 28])
])

In [ ]:
#Bulid model in Sequential

stacked_autoencoder = keras.Sequential([encoder, decoder])

Note that we use binary cross entropy loss in stead of categorical cross entropy. The reason for that is because we are not classifying latent vectors to belong to a particular class, we do not even have classes!, but rather are trying to predict whether a pixel should be activated or not.

In [ ]:
# Define the optimizer with a specific learning rate
optimizer = Adam(learning_rate=0.001)

# Compile the stacked autoencoder model with the optimizer
stacked_autoencoder.compile(optimizer=optimizer,
                            loss='binary_crossentropy',  # Use appropriate loss function for reconstruction
                            metrics=['accuracy'])

# Print model summary
stacked_autoencoder.summary()

Notice how the $x$ and $y$, both $x$, `x_train` if you like, are equal:

In [ ]:
#Train the model
stacked_autoencoder.fit(x_train, x_train, epochs=15, batch_size=10 ,validation_data=[x_test, x_test])

In [ ]:
import matplotlib.pyplot as plt

# Assuming stacked_autoencoder is your trained autoencoder model
# Assuming x_test contains your test data

# Set the figure size
plt.figure(figsize=(20, 5))

# Loop through and plot original and reconstructed images
for i in range(8):
    # Original image
    original = x_test[i].reshape(28, 28)

    # Reconstructed image
    reconstructed = stacked_autoencoder.predict(x_test[i].reshape(1, 28, 28))[0].reshape(28, 28)

    # Plotting original image
    plt.subplot(2, 8, i + 1)
    plt.imshow(original, cmap='binary')  # Fixed cmap='binary' instead of 'binarry'
    plt.title('Original')


    # Plotting reconstructed image
    plt.subplot(2, 8, 8 + i + 1)
    plt.imshow(reconstructed, cmap='binary')  # Fixed cmap='binary' instead of 'binarry'
    plt.title('Reconstructed')

# Adjust layout and display plot
plt.tight_layout()
plt.show()


In [ ]:
# Set the index of the image you want to visualize
i = 0

# Set up the figure and subplots
plt.figure(figsize=(15, 5))

# Plot the original image
plt.subplot(1, 3, 1)
plt.imshow(x_test[i], cmap='binary')
plt.title('Original Image')


# Plot the latent space representation
plt.subplot(1, 3, 2)
latent_space = encoder.predict(x_test[i].reshape(1, 28, 28))
plt.imshow(latent_space, cmap='binary')  # Assuming latent_space shape is (1, 30)
plt.title('Latent Space Representation')


# Reconstruct the image from the latent space representation
reconstructed = decoder.predict(latent_space)
plt.subplot(1, 3, 3)
plt.imshow(reconstructed.reshape(28, 28), cmap='binary')
plt.title('Reconstructed Image')


# Show the plot
plt.tight_layout()
plt.show()


In [ ]:
# Calculate sparsity_lower: This calculates the proportion of zero elements in a 28x28 matrix
# where 30 elements are zero.
sparsity_lower = 30 / (28 * 28)

# Calculate sparsity_higher: This calculates the proportion of non-zero elements in the same
# 28x28 matrix.
sparsity_higher = 1 - sparsity_lower

# Display the results
print("Sparsity Lower:", sparsity_lower)
print("Sparsity Higher:", sparsity_higher)


That's 96.2% compression!